# Exploring Metrics for Business Popularity

In [1]:
import json
import pandas as pd
import numpy as np
import math
import datetime

In [2]:
business_df = pd.read_json('./yelp/yelp_academic_dataset_business.json', lines=True)
business_df.drop(columns=['address', 'postal_code', 'name', 'stars', 'review_count'], inplace=True)
# business_df.dropna(inplace=True)  # the nans are for attributes and opening hours: may not be integral part
business_df.set_index("business_id", inplace=True)
print(len(business_df))

checkins_df = pd.read_json('./yelp/yelp_academic_dataset_checkin.json', lines=True)
print(len(checkins_df))

reviews_df = pd.read_json('yelp/yelp_academic_dataset_review.json', lines=True)
reviews_df.drop(columns=["text", "user_id", "review_id"], inplace=True)
reviews_df["date"] = pd.to_datetime(reviews_df["date"])

tips_df = pd.read_json('yelp/yelp_academic_dataset_tip.json', lines=True)
print(len(tips_df))

YEAR_RANGE = (datetime.datetime(2017, 1, 1, 0, 0), datetime.datetime(2018, 12, 31, 23, 59))

160585
138876
1162119


In [3]:
def filter_date(date):
    if date < YEAR_RANGE[0] or date > YEAR_RANGE[1]:
        return False
    return True

def calculate_checkin_count(row):
    dates = [datetime.datetime.strptime(d.strip(), "%Y-%m-%d %H:%M:%S") for d in row.split(",")]
    dates = list(filter(filter_date, dates))

    return len(dates)

def start_date(row):
    dates = row.split(",")
    return datetime.datetime.strptime(dates[0].strip(), "%Y-%m-%d %H:%M:%S")

def last_date(row):
    dates = row.split(",")
    return datetime.datetime.strptime(dates[-1].strip(), "%Y-%m-%d %H:%M:%S")

checkins_df.set_index("business_id", inplace=True)
checkins_df['checkin_count'] = checkins_df["date"].apply(calculate_checkin_count)
checkins_df['first_checkin'] = checkins_df["date"].apply(start_date)
checkins_df['last_checkin'] = checkins_df["date"].apply(last_date)
checkins_df.head()

,date,checkin_count,first_checkin,last_checkin
business_id,,,,
--0r8K_AQ4FZfLsX3ZYRDA,2017-09-03 17:13:59,1,2017-09-03 17:13:59,2017-09-03 17:13:59
--0zrn43LEaB4jUWTQH_Bg,"2010-10-08 22:21:20, 2010-11-01 21:29:14, 2010...",0,2010-10-08 22:21:20,2011-08-29 19:01:31
--164t1nclzzmca7eDiJMw,"2010-02-26 02:06:53, 2010-02-27 08:00:09, 2010...",0,2010-02-26 02:06:53,2013-11-25 02:51:33
--2aF9NhXnNVpDV0KS3xBQ,"2014-11-03 16:35:35, 2015-01-30 18:16:03, 2015...",2,2014-11-03 16:35:35,2020-12-29 16:22:00
--2mEJ63SC_8_08_jGgVIg,"2010-12-15 17:10:46, 2013-12-28 00:27:54, 2015...",0,2010-12-15 17:10:46,2016-06-11 19:56:11


In [4]:
# last_date >= 2017, first_date < 2018.01.01 (at least opened before 2018)
checkins_df = checkins_df[(checkins_df["last_checkin"] >= YEAR_RANGE[0]) & (checkins_df["first_checkin"] < YEAR_RANGE[0] + datetime.timedelta(days=365))]
checkins_df.drop(columns="date", inplace=True)

In [5]:
# merge checkin
business_df = pd.merge(business_df, checkins_df, on='business_id')

len(business_df)

87702

## Process Reviews

In [6]:
reviews_data = reviews_df.groupby(by='business_id').agg(review_count=("date", lambda x: x[(x >= YEAR_RANGE[0]) & (x <= YEAR_RANGE[1])].count()),
                                                        first_review=("date", lambda x: x.min()),
                                                        last_review=("date", lambda x: x.max()))

reviews_data.head()

,review_count,first_review,last_review
business_id,,,
--0DF12EMHYI8XIgoFha6A,1,2014-11-25 20:35:03,2020-12-21 16:40:17
--0r8K_AQ4FZfLsX3ZYRDA,2,2017-09-03 17:15:48,2020-04-15 13:33:08
--0zrn43LEaB4jUWTQH_Bg,0,2011-05-03 22:15:28,2012-03-26 22:55:52
--164t1nclzzmca7eDiJMw,0,2009-03-17 01:53:31,2014-06-09 21:27:05
--2aF9NhXnNVpDV0KS3xBQ,1,2008-08-08 17:13:26,2021-01-01 14:39:44


In [7]:
reviews_data = reviews_data[(reviews_data["last_review"] >= YEAR_RANGE[0]) & (reviews_data["first_review"] < YEAR_RANGE[0] + datetime.timedelta(days=365))]

In [8]:
business_df = pd.merge(business_df, reviews_data, on='business_id')
len(business_df)

85797

In [9]:
def round_star(raw_star):

    i = raw_star // 1
    f = raw_star % 1
    if f >= 0.75:
        ans = i + 1
    elif f >=0.25:
        ans = i + 0.5
    else:
        ans = i

    return ans

ranged_reviews_df = reviews_df[(reviews_df["date"] >= YEAR_RANGE[0]) & (reviews_df["date"] <= YEAR_RANGE[1])]

business_df["raw_stars"] = ranged_reviews_df.groupby("business_id").mean()["stars"]
business_df["stars"] = business_df["raw_stars"].apply(round_star)

business_df[["stars", "raw_stars"]]

,stars,raw_stars
business_id,,
6iYb2HFDywm3zjuRg0shjw,3.5,3.714286
tCbdrRPZA0oiIYSmHG3J0w,3.5,3.500000
bvN78flM8NLprQ1a1y5dRg,5.0,5.000000
oaepsyvc0J17qwi8cfrOWg,3.0,3.200000
PE9uqAjdw0E4-8mjGl3wVA,5.0,5.000000
...,...,...
m1Qw53OgkxrWu59oddVf9A,3.5,3.666667
D2mHoIDXx9N8mS1pGoKV9Q,2.5,2.333333
bQX-kwVTyZgcdZGEPzce6Q,5.0,5.000000


## Process tips

In [10]:
tips_data = tips_df.groupby(by='business_id').agg(tip_count=("date", lambda x: x[(x >= YEAR_RANGE[0]) & (x <= YEAR_RANGE[1])].count()),
                                                  first_tip=("date", lambda x: x.min()),
                                                  last_tip=("date", lambda x: x.max()))


In [11]:
tips_data = tips_data[(tips_data["last_tip"] >= YEAR_RANGE[0]) & (tips_data["first_tip"] < YEAR_RANGE[0] + datetime.timedelta(days=365))]

In [12]:
business_df = pd.merge(business_df, tips_data, on='business_id')
len(business_df)

47979

## Put all the dates and counts together

In [13]:
business_df["first_date"] = business_df[["first_checkin", "first_review", "first_tip"]].min(axis=1)
business_df["last_date"] = business_df[["last_checkin", "last_review", "last_tip"]].max(axis=1)

business_df["visit_count"] = business_df[["checkin_count", "review_count", "tip_count"]].sum(axis=1)

In [14]:
business_df["is_open_year_after"] = business_df["last_date"] > YEAR_RANGE[1]

## Restaurants

In [15]:
restaurants_df = business_df.loc[business_df["categories"].str.contains("Restaurants") | business_df["categories"].str.contains("Food")]
print(f"There are {len(restaurants_df)} restaurants in the dataset")

restaurants_df.head(3)

There are 30094 restaurants in the dataset


,city,state,latitude,longitude,is_open,attributes,categories,hours,checkin_count,first_checkin,...,last_review,raw_stars,stars,tip_count,first_tip,last_tip,first_date,last_date,visit_count,is_open_year_after
business_id,,,,,,,,,,,,,,,,,,,,,
6iYb2HFDywm3zjuRg0shjw,Boulder,CO,40.017544,-105.283348,1,"{'RestaurantsTableService': 'True', 'WiFi': 'u...","Gastropubs, Food, Beer Gardens, Restaurants, B...","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'...",79,2017-09-10 04:48:12,...,2021-01-22 05:20:38,3.714286,3.5,4,2017-09-09 04:42:34,2019-09-17 04:30:53,2017-09-09 04:42:34,2021-01-22 05:20:38,132,True
tCbdrRPZA0oiIYSmHG3J0w,Portland,OR,45.588906,-122.593331,1,"{'RestaurantsTakeOut': 'True', 'RestaurantsAtt...","Salad, Soup, Sandwiches, Delis, Restaurants, C...","{'Monday': '5:0-18:0', 'Tuesday': '5:0-17:0', ...",181,2010-04-22 05:31:33,...,2020-08-06 09:00:59,3.500000,3.5,4,2011-05-28 02:06:25,2019-06-25 18:21:12,2010-03-09 16:02:04,2021-01-21 17:55:35,209,True
D4JtQNTI4X3KcbzacDJsMw,Vancouver,BC,49.251342,-123.101333,1,"{'GoodForKids': 'True', 'Alcohol': 'u'none'', ...","Restaurants, Thai","{'Monday': '17:0-21:0', 'Tuesday': '17:0-21:0'...",42,2010-11-06 02:53:03,...,2020-12-24 21:24:42,3.678571,3.5,3,2010-12-02 21:10:51,2019-09-01 03:01:29,2010-09-26 04:03:35,2021-01-23 01:43:50,73,True


In [16]:
restaurants_df.drop(columns=["first_checkin", "first_review", "first_tip", "last_checkin", "last_review", "last_tip"], inplace=True)
restaurants_df.head(3)

/data/tianqing/Downloads/ENTER/envs/sh_DISCOS/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,city,state,latitude,longitude,is_open,attributes,categories,hours,checkin_count,review_count,raw_stars,stars,tip_count,first_date,last_date,visit_count,is_open_year_after
business_id,,,,,,,,,,,,,,,,,
6iYb2HFDywm3zjuRg0shjw,Boulder,CO,40.017544,-105.283348,1,"{'RestaurantsTableService': 'True', 'WiFi': 'u...","Gastropubs, Food, Beer Gardens, Restaurants, B...","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'...",79,49,3.714286,3.5,4,2017-09-09 04:42:34,2021-01-22 05:20:38,132,True
tCbdrRPZA0oiIYSmHG3J0w,Portland,OR,45.588906,-122.593331,1,"{'RestaurantsTakeOut': 'True', 'RestaurantsAtt...","Salad, Soup, Sandwiches, Delis, Restaurants, C...","{'Monday': '5:0-18:0', 'Tuesday': '5:0-17:0', ...",181,24,3.500000,3.5,4,2010-03-09 16:02:04,2021-01-21 17:55:35,209,True
D4JtQNTI4X3KcbzacDJsMw,Vancouver,BC,49.251342,-123.101333,1,"{'GoodForKids': 'True', 'Alcohol': 'u'none'', ...","Restaurants, Thai","{'Monday': '17:0-21:0', 'Tuesday': '17:0-21:0'...",42,28,3.678571,3.5,3,2010-09-26 04:03:35,2021-01-23 01:43:50,73,True


In [17]:
restaurants_df.to_csv("datasets/2017-2018_restaurants.csv")

## Explanation on columns

The dataframe consists of data of restaurants that were already open before or in 2017 and had some activity during 2017-2018 period. Only the data entry between this period was considered; i.e. conceptually, all restaurants are assumed that they started in 2017.

* is_open: whether the restaurant is open in 2021.03
* is_open_year_later: whether the restaurant was open the year later from the timeframe we considered. i.e. if there was activity record after 2018.12.31 11:59.
* checkin, review, tip count: the count of checkin, review, and tip only from 2017-2018.
* visit_count: sum of checkin, review, and tip count. Total number of visits in 2017-2018.
* stars: average of star ratings from the reviews only from 2017-2018.
* first_date, last_date: first and last date any of the activity occured, from the WHOLE timeframe (not just within 2017-2018).
    
    ```
    first date < 2018.01.01, last date >= 2017.01.01
    ```

In [18]:
restaurants_df[["first_date", "last_date"]].describe()

<ipython-input-18-43f91036fd92>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  restaurants_df[["first_date", "last_date"]].describe()
<ipython-input-18-43f91036fd92>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  restaurants_df[["first_date", "last_date"]].describe()


,first_date,last_date
count,30094,30094
unique,30093,30020
top,2011-02-03 00:13:31,2021-01-22 23:19:27
freq,2,3
first,2004-10-14 02:57:52,2017-01-18 23:07:21
last,2017-12-28 19:49:17,2021-01-28 15:38:54
